In [3]:
import os
import numpy as np
import pandas as pd

from library.Data_Preprocessing import Balance_Ratio 

os.chdir('C:/Users/user/Desktop/Darui_R08621110') 
os.getcwd()

'C:\\Users\\user\\Desktop\\Darui_R08621110'

## 

### identfy all kinds of dimensional overlapping data

In [4]:
def Corner(D, label = 'GB'):
    
    Data = D.copy()
    Data['cb'] = Data[Data.columns[1:-1]].apply(lambda x: '_'.join(x.astype(str)), axis = 1)
    Data[f'{label}_mean'] = Data.groupby('cb')[label].transform('mean')
    Data[f'{label}_count'] = Data.groupby('cb')[label].transform('count')
    Corner_overlap = Data.copy() 
    
    return Corner_overlap


def Kind(Corner_overlap, label = 'GB'):

    Data = Corner_overlap
    
    Kinds_0 = Data[Data[f'{label}_mean'] == 0].sort_values(f'{label}_count') # represent all data in the same cb are good  
    Kinds_1 = Data[Data[label] == 1].sort_values(f'{label}_count') # represent bad data
    Kinds = pd.concat([Kinds_1, Kinds_0]) # reprecent all types of 'cb'
    Kinds_after_duplicate = Kinds.drop_duplicates(subset = ['cb']).reset_index(drop = True) 
    # each 'cb' only keep one data(the first)
    
    return Kinds_after_duplicate


def Dictionary_Build(Data):
    
    Corner_overlap = Corner(Data)
    Kinds = Kind(Corner_overlap).copy()

    ## reorder cols
    cols = Kinds.columns.tolist()
    cols = cols[0:1] + cols[-4:] 
    Dictionary = Kinds[cols] #select id, GB, cb, GB_mean, GB_count
    Dictionary['G_count'] = 0 # add new column 
    Dictionary=Dictionary.reset_index(drop=True)
    for i in range(len(Dictionary)):
        Dictionary['G_count'][i] = Dictionary.GB_count[i] - Dictionary.GB[i] # represent??
    
    return Dictionary

### relabel overlapping data

In [5]:
def Remove_SD(Data, count = 1, label = 'GB'):
    
    Corner_Overlap = Data
    RSD = pd.concat([Data[Data[label] == 1], Data[(Data[label] == 0) & (Data[f'{label}_count'] > count)]]) 
    # remove good data that only show once
    return RSD


def Corner_Blend(Data, ratio = 0.002, label = 'GB'):
    
    D_1 = Data[(Data[f'{label}_mean'] <= 1) & (Data[f'{label}_mean'] >= ratio)].sort_values(f'{label}_count') # not relabel case
    D_1[[label]] = 1  #if G_mean > ratio ==> relabel all data as bad
    D_2 = Data[(Data[f'{label}_mean'] < ratio)].sort_values(f'{label}_count')
    D_2[[label]] = 0  # otherwise relabel all data as good
    Training_new = pd.concat([D_1,D_2]).iloc[:,:-3]
    
    return Training_new

## 

### loading training data

In [32]:
##### training data of each month #####
training_month = [2, 3, 4]
train_runhist = {}
for i in training_month:
    train_runhist[f'm{i}'] = pd.read_csv(f'train_runhist_m{i}.csv').iloc[:, 1:]
    print(f'Dimension of month {i}:', train_runhist[f'm{i}'].shape)

##### training & testing data #####
test_runhist = pd.read_csv('test_runhist.csv').iloc[:, 1:]
print('\nDimension of training data:', train_runhist['all'].shape,
      '\nDimension of testing data:', test_runhist.shape)

Dimension of month 2: (38022, 97)
Dimension of month 3: (60607, 108)
Dimension of month 4: (57742, 103)

Dimension of training data: (156371, 116) 
Dimension of testing data: (48654, 116)


### relabel the training data by month

In [44]:
##### training data of each month #####
training_month = [2, 3, 4]

overlap = {}
new_runhist = pd.DataFrame()
for i in training_month:
    overlap[f'm{i}'] = Corner(train_runhist[f'm{i}'])
    new_runhist[f'm{i}'] = Corner_Blend(overlap[f'm{i}'], 1/10000)
    new_runhist[f'm{i}'].to_csv(f'relabel_runhist_m{i}.csv')
    print('Month {i} after overlap relabel:', new_runhist[f'm{i}'].shape, 'balance ratio:', 
          Balance_Ratio(new_runhist[f'm{i}']), '# total bad:', sum(new_runhist[f'm{i}'].GB))
    
##### the whole training data #####
new_train = pd.DataFrame()
for i in training_month:
    new_train = pd.concat([new_train, new_runhist[f'm{i}']])
print('\nAll training data after overlap relabel:', new_train.shape, ', balance ratio:', Balance_Ratio(new_train), 
          ', # total bad:', sum(new_train.GB))

##### the whole data #####
new_all = pd.concat([new_train, test_runhist], axis = 0)
print('All runhist data after overlap relabel:', new_all.shape, ', balance ratio:', Balance_Ratio(new_all), 
          ', # total bad:', sum(new_all.GB))

Month {i} after overlap relabel: (38022, 97) balance ratio: 863.0 # total bad: 44
Month {i} after overlap relabel: (60607, 108) balance ratio: 550.0 # total bad: 110
Month {i} after overlap relabel: (57742, 103) balance ratio: 549.0 # total bad: 105

All training data after overlap relabel: (156371, 116) , balance ratio: 578.0 , # total bad: 270
All runhist data after overlap relabel: (205025, 116) , balance ratio: 561.0 , # total bad: 365


### cauculate the number of combination types 

In [45]:
##### training data of each month #####
kinds = {}
for i in training_month:
    kinds[f'm{i}'] = Kind(overlap[f'm{i}'])
    kinds[f'm{i}'].to_csv(f'kind_m{i}.csv')
    print('Number of kinds in month {i}:', len(kinds[f'm{i}']))

##### the whole training data & testing data #####
train_kinds = Kind(train_overlap)
train_kinds.to_csv('kind.csv')
print('\nNumber of kinds in all training data:', len(train_kinds))

test_overlap = Corner(test_runhist)
test_kinds = Kind(test_overlap)
all_overlap = Corner(new_all)
all_kinds = Kind(all_overlap)

print('Number of kinds in all testing data:', len(test_kinds))
print('Number of kinds in all runhist data:', len(all_kinds))

Number of kinds in month {i}: 24537
Number of kinds in month {i}: 36690
Number of kinds in month {i}: 34462

Number of kinds in all training data: 91515
Number of kinds in all testing data: 29967
Number of kinds in all runhist data: 121482
